<a href="https://colab.research.google.com/github/nikhil991gupta/Data-Analysis/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('data.csv')


In [3]:
missing_values = df.isnull().sum()


In [5]:
print("Original Columns:", df.columns)


Original Columns: Index(['order_id', 'order_date', 'sku_id', 'product_name', 'final_ak_price',
       'units_sold', 'quantity', 'customer_id', 'placed_gmv'],
      dtype='object')


In [6]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


In [7]:
print("Cleaned Columns:", df.columns)


Cleaned Columns: Index(['order_id', 'order_date', 'sku_id', 'product_name', 'final_ak_price',
       'units_sold', 'quantity', 'customer_id', 'placed_gmv'],
      dtype='object')


In [8]:
print("Missing values:\n", df.isnull().sum())


Missing values:
 order_id          0
order_date        0
sku_id            0
product_name      0
final_ak_price    0
units_sold        0
quantity          0
customer_id       0
placed_gmv        0
dtype: int64


In [12]:
numeric_columns = ['final_ak_price', 'units_sold', 'quantity', 'placed_gmv']


In [13]:
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')


In [14]:
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())


In [15]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


In [16]:
df = df[(df['final_ak_price'] > 0) & (df['units_sold'] > 0) & (df['quantity'] > 0)]


In [17]:
df['calculated_gmv'] = df['final_ak_price'] * df['units_sold'] * df['quantity']


In [18]:
df = df[df['placed_gmv'] == df['calculated_gmv']]


In [19]:
df.drop(columns=['calculated_gmv'], inplace=True)


In [20]:
df.to_csv('cleaned_dataset.csv', index=False)


In [21]:
print("Cleaned data preview:\n", df.head())


Cleaned data preview:
    order_id          order_date      sku_id  \
1    119566 2022-05-05 11:47:05  622854da66   
2    119566 2022-05-05 11:47:05  61d439f99a   
3    119566 2022-05-05 11:47:05  60dec56d3e   
4    119566 2022-05-05 11:47:05  61021fdb38   
5    119566 2022-05-05 11:47:05  6107de458b   

                                        product_name  final_ak_price  \
1     Navratna Extra Thanda Ayurvedic Hair Oil 2.5ml            0.69   
2                           Dabur Amla Hair Oil 30ml            8.28   
3    Garnier Bright Complete Vitamin C Face Wash 50g           61.34   
4           Patanjali's Saundarya Aloe Vera Gel 60ml           38.52   
5  Oxylife Natural Radiance 5 Creme Bleach With A...           35.79   

   units_sold  quantity  customer_id  placed_gmv  
1         240         2        99710      331.20  
2          20         1        99710      165.60  
3           6         1        99710      368.04  
4          12         2        99710      924.48  
5     

In [22]:
!pip install pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=0fc875342898c5daffc4668b232252709453125729550f60f4193ce7cba18c6f
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [23]:
import pandas as pd
import pandasql as ps


In [24]:
df = pd.read_csv('cleaned_dataset.csv')


In [25]:
def run_query(query):
    return ps.sqldf(query, globals())

# Example SQL Query 1: Find the average order value, total number of orders, first order date, last order date, first order value, and last order value of a customer
query1 = """
    SELECT
        customer_id,
        AVG(placed_gmv) AS avg_order_value,
        COUNT(order_id) AS total_orders,
        MIN(order_date) AS first_order_date,
        MAX(order_date) AS last_order_date,
        (SELECT placed_gmv FROM df WHERE customer_id = t.customer_id ORDER BY order_date ASC LIMIT 1) AS first_order_value,
        (SELECT placed_gmv FROM df WHERE customer_id = t.customer_id ORDER BY order_date DESC LIMIT 1) AS last_order_value
    FROM df t
    GROUP BY customer_id
"""

result1 = run_query(query1)
print(result1)

     customer_id  avg_order_value  total_orders     first_order_date  \
0            376         2435.856             5  2022-05-07 16:49:10   
1            750          337.680             2  2022-05-13 09:01:42   
2           1209         2138.400             1  2022-05-04 11:23:59   
3           1957         2671.300             3  2022-05-25 23:31:33   
4           2349         1122.360             2  2022-05-18 10:27:15   
..           ...              ...           ...                  ...   
944       132137         1260.000             2  2022-06-20 14:01:50   
945       132428         1198.500             2  2022-06-21 19:34:19   
946       133057         2848.000             1  2022-06-21 23:40:57   
947       133647         5190.000             1  2022-06-23 13:15:11   
948       133801         5337.000             1  2022-06-23 10:39:47   

         last_order_date  first_order_value  last_order_value  
0    2022-05-13 22:46:40            1156.68           5220.00  
1    20

In [26]:
# Example SQL Query 2: List the top 20 products in the Popular Products category based on units sold
query2 = """
    SELECT
        product_name,
        SUM(units_sold) AS total_units_sold
    FROM df
    GROUP BY product_name
    ORDER BY total_units_sold DESC
    LIMIT 20
"""

result2 = run_query(query2)
print(result2)


                                         product_name  total_units_sold
0              Navratna Cool Ayurvedic Hair Oil 2.7ml              4704
1      Navratna Extra Thanda Ayurvedic Hair Oil 2.5ml              3360
2                    Dettol Original Bathing Soap 40g              3091
3           Surf Excel Easy Wash Detergent Powder 80g              2436
4   Head & Shoulders Anti Dandruff 2in1 Shampoo pl...              2400
5                       Bru Instant Coffee Pouch 2.2g              2304
6                          Kissan Mixed Fruit Jam 11g              1920
7   Set Wet Styling Gel Gravity Resist Vertical Ho...              1512
8         Closeup Everfresh+ Triple Fresh Formula 23g              1296
9          Cadbury Oreo Original Vanilla Creme  46.3g              1128
10                            MamyPoko Pants Small 1U              1080
11             Kissan Chotu Fresh Tomato Ketchup 130g              1020
12                                Prabhuji Bhujia 32g           

In [27]:
# Example SQL Query 3: Find the top 3 articles for each customer in terms of placed_gmv
query3 = """
    SELECT *
    FROM (
        SELECT
            customer_id,
            product_name,
            placed_gmv,
            ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY placed_gmv DESC) AS rank
        FROM df
    ) ranked
    WHERE rank <= 3
"""

result3 = run_query(query3)
print(result3)

      customer_id                                    product_name  placed_gmv  \
0             376              Tata Tea Premium India ki Chai 1kg     5220.00   
1             376              Tata Tea Premium India ki Chai 1kg     3480.00   
2             376  Odonil Bathroom Air Freshener 200g Buy 3 Get 1     1482.60   
3             750         Mai Ghar Disinfectant Toilet Cleaner 1L      450.00   
4             750                      Surf Excel Mineral Bar 84g      225.36   
...           ...                                             ...         ...   
2162       132428         Lizol Disinfectant Cleaner Citrus 500ml     1557.00   
2163       132428                       Dabur Red Paste 12U X 42g      840.00   
2164       133057        Clinic Plus Strong & Long Sachet 960X6ml     2848.00   
2165       133647              Tata Tea Premium India ki Chai 1kg     5190.00   
2166       133801      Surf Excel Easy Wash Detergent Powder 500g     5337.00   

      rank  
0        1  
1